In [1]:
# 라이브러리
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import calendar
from datetime import datetime, timedelta

import os


%matplotlib inline

# 경고메세지 팝업 끄기
import warnings
warnings.filterwarnings("ignore")

In [ ]:
pd.set_option('display.max_columns',None)
pd.set_option('display.max_rows',None)

# 데이터 불러오기

In [2]:
df=pd.read_csv('fda_train.csv')
df

,store_id,card_id,card_company,transacted_date,transacted_time,installment_term,region,type_of_business,amount
0,0,0,b,2016-06-01,13:13,0,NaN,기타 미용업,1857.142857
1,0,1,h,2016-06-01,18:12,0,NaN,기타 미용업,857.142857
2,0,2,c,2016-06-01,18:52,0,NaN,기타 미용업,2000.000000
3,0,3,a,2016-06-01,20:22,0,NaN,기타 미용업,7857.142857
4,0,4,c,2016-06-02,11:06,0,NaN,기타 미용업,2000.000000
...,...,...,...,...,...,...,...,...,...
6556608,2136,4663855,d,2019-02-28,23:20,0,제주 제주시,기타 주점업,-4500.000000
6556609,2136,4663855,d,2019-02-28,23:24,0,제주 제주시,기타 주점업,4142.857143
6556610,2136,4663489,a,2019-02-28,23:24,0,제주 제주시,기타 주점업,4500.000000
6556611,2136,4663856,d,2019-02-28,23:27,0,제주 제주시,기타 주점업,571.428571


# 데이터 전처리

## amount 데이터 타입 변경

In [3]:
df['amount'] = df['amount'].astype(int)

## 날짜 및 지역 관련 데이터 처리

In [4]:
df["date"] = df["transacted_date"]+ " " + df["transacted_time"]

In [5]:
df["date"]  = pd.to_datetime(df["date"])

### 연, 월, 요일, 시간 , 휴일, 분기, 계절, 지역 데이터 분리

In [6]:
df["year"] = df["date"].dt.year
df["month"] = df["date"].dt.month
df["dayofweek"] = df["date"].dt.weekday # 0-월요일, 1-화요일
df["hour"] = df["date"].dt.hour

In [7]:
def holidays(dayofweek):
    if dayofweek in [0,1,2,3,4]:
        return "0"
    elif dayofweek in [5,6]:
        return "1"

df["holidays"] = df.dayofweek.apply(holidays)

In [8]:
# 분기
df["quarter"] = df["date"].dt.quarter

In [9]:
# 계절 
def season(month):
    season = 'winter'
    if 3 <= month <=5:
        season = 'spring'
    elif 6<= month <=8:
        season = 'summer'
    elif 9<= month <=11:
        season = 'fall'
    
    return season

df["season"] = df.month.apply(season)

In [10]:
df["sido"]= df["region"].str.split(" ").str[0]
df["city"]= df["region"].str.split(" ").str[1]

In [11]:
# 지역재분류
def area(sido):
    area = '제주'
    if sido == '서울':
        area = '서울'
    elif sido == '강원':
        area = '강원'
    elif sido in[ '경기','인천']:
        area = '경기인천'
    elif sido in ['대전','세종','충남','충북']:
        area = '호서'
    elif sido in ['부산','대구','울산','경남','경북']:
        area = '영남'
    elif sido in ['광주','전남','전북']:
        area = '호남'
    
    
    return area

df["area"] = df.sido.apply(area)

In [ ]:
df.info()

In [12]:
df.head(3)

,store_id,card_id,card_company,transacted_date,transacted_time,installment_term,region,type_of_business,amount,date,year,month,dayofweek,hour,holidays,quarter,season,sido,city,area
0,0,0,b,2016-06-01,13:13,0,NaN,기타 미용업,1857,2016-06-01 13:13:00,2016,6,2,13,0,2,summer,NaN,NaN,제주
1,0,1,h,2016-06-01,18:12,0,NaN,기타 미용업,857,2016-06-01 18:12:00,2016,6,2,18,0,2,summer,NaN,NaN,제주
2,0,2,c,2016-06-01,18:52,0,NaN,기타 미용업,2000,2016-06-01 18:52:00,2016,6,2,18,0,2,summer,NaN,NaN,제주


## 특일 데이터 적용

In [13]:
do = pd.read_csv('holiday.csv')
do

,일자 및 요일,요일구분,비고
0,2016-01-01,금요일,신정
1,2016-02-08,월요일,설날
2,2016-02-09,화요일,설날
3,2016-02-10,수요일,설날
4,2016-03-01,화요일,삼일절
...,...,...,...
56,2019-09-13,금요일,추석
57,2019-10-03,목요일,개천절
58,2019-10-09,수요일,한글날
59,2019-12-25,수요일,성탄절


x=do[do["비고"]=='연말휴장일'].index
do = do.drop(x)

In [14]:
### 2019년 3월 이후 데이터 삭제 (df데이터와 일수 맞추기 ) # 위에 것 안해도 됨
do = do.truncate(after = 49)
do = do.truncate(before = 8)

In [15]:
do.rename(columns={'일자 및 요일':'transacted_date'}, inplace=True)

In [16]:
do.rename(columns={'요일구분':'day'}, inplace=True)

In [17]:
do["date"] = pd.to_datetime(do["transacted_date"])


In [18]:
do["dayofweek"] = do["date"].dt.weekday
do

,transacted_date,day,비고,date,dayofweek
8,2016-06-06,월요일,현충일,2016-06-06,0
9,2016-08-15,월요일,광복절,2016-08-15,0
10,2016-09-14,수요일,추석,2016-09-14,2
11,2016-09-15,목요일,추석,2016-09-15,3
12,2016-09-16,금요일,추석,2016-09-16,4
13,2016-10-03,월요일,개천절,2016-10-03,0
14,2017-01-27,금요일,설날,2017-01-27,4
15,2017-01-30,월요일,설날(대체휴일),2017-01-30,0
16,2017-03-01,수요일,삼일절,2017-03-01,2
17,2017-05-01,월요일,근로자의날,2017-05-01,0


In [19]:
do["day"].unique()

array(['월요일   ', '수요일   ', '목요일   ', '금요일   ', '화요일   '], dtype=object)

In [20]:
def holidays(dayofweek):
    if dayofweek in [0,1,2,3,4]:
        return "1"
    else:
        return "0"

do["holidays"] = do.dayofweek.apply(holidays)
do.head()

,transacted_date,day,비고,date,dayofweek,holidays
8,2016-06-06,월요일,현충일,2016-06-06,0,1
9,2016-08-15,월요일,광복절,2016-08-15,0,1
10,2016-09-14,수요일,추석,2016-09-14,2,1
11,2016-09-15,목요일,추석,2016-09-15,3,1
12,2016-09-16,금요일,추석,2016-09-16,4,1


In [ ]:
do.info()

In [21]:
do = do.drop(['day','비고', 'date', 'dayofweek'], axis = 1)

In [ ]:
do

In [22]:
df= pd.merge(df,do, how='outer', on='transacted_date')
df

,store_id,card_id,card_company,transacted_date,transacted_time,installment_term,region,type_of_business,amount,date,...,dayofweek,hour,holidays_x,quarter,season,sido,city,area,비고,holidays_y
0,0,0,b,2016-06-01,13:13,0,NaN,기타 미용업,1857,2016-06-01 13:13:00,...,2,13,0,2,summer,NaN,NaN,제주,NaN,NaN
1,0,1,h,2016-06-01,18:12,0,NaN,기타 미용업,857,2016-06-01 18:12:00,...,2,18,0,2,summer,NaN,NaN,제주,NaN,NaN
2,0,2,c,2016-06-01,18:52,0,NaN,기타 미용업,2000,2016-06-01 18:52:00,...,2,18,0,2,summer,NaN,NaN,제주,NaN,NaN
3,0,3,a,2016-06-01,20:22,0,NaN,기타 미용업,7857,2016-06-01 20:22:00,...,2,20,0,2,summer,NaN,NaN,제주,NaN,NaN
4,1,1931,b,2016-06-01,13:31,0,NaN,NaN,1071,2016-06-01 13:31:00,...,2,13,0,2,summer,NaN,NaN,제주,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6556608,2136,4659988,f,2017-10-04,23:32,0,제주 제주시,기타 주점업,9571,2017-10-04 23:32:00,...,2,23,0,4,fall,제주,제주시,제주,추석,1
6556609,2136,4659465,f,2017-10-04,23:41,0,제주 제주시,기타 주점업,7000,2017-10-04 23:41:00,...,2,23,0,4,fall,제주,제주시,제주,추석,1
6556610,2136,4659989,d,2017-10-04,23:47,0,제주 제주시,기타 주점업,5000,2017-10-04 23:47:00,...,2,23,0,4,fall,제주,제주시,제주,추석,1
6556611,2136,4659710,g,2017-10-04,23:53,0,제주 제주시,기타 주점업,3785,2017-10-04 23:53:00,...,2,23,0,4,fall,제주,제주시,제주,추석,1


### merge 관련 결측치 정리

In [23]:
df["비고"] = df["비고"].fillna('0')


In [24]:
df["holidays_y"] = df["holidays_y"].fillna('0')
df.sample(3)

,store_id,card_id,card_company,transacted_date,transacted_time,installment_term,region,type_of_business,amount,date,...,dayofweek,hour,holidays_x,quarter,season,sido,city,area,비고,holidays_y
1319779,672,1336041,a,2017-04-08,18:50,0,강원 속초시,의복 소매업,24000,2017-04-08 18:50:00,...,5,18,1,2,spring,강원,속초시,강원,,0
3463326,600,1167573,b,2018-04-06,22:57,0,서울 구로구,NaN,38285,2018-04-06 22:57:00,...,4,22,0,2,spring,서울,구로구,서울,,0
2624545,1400,3025863,c,2017-11-09,15:04,0,NaN,NaN,1928,2017-11-09 15:04:00,...,3,15,0,4,fall,NaN,NaN,제주,,0


In [25]:
df = df.astype({'holidays_x':'int', 'holidays_y':'int'})

In [ ]:
df.info()

In [26]:
df["holi_sum"] = df['holidays_x'] + df['holidays_y']
df

,store_id,card_id,card_company,transacted_date,transacted_time,installment_term,region,type_of_business,amount,date,...,hour,holidays_x,quarter,season,sido,city,area,비고,holidays_y,holi_sum
0,0,0,b,2016-06-01,13:13,0,NaN,기타 미용업,1857,2016-06-01 13:13:00,...,13,0,2,summer,NaN,NaN,제주,,0,0
1,0,1,h,2016-06-01,18:12,0,NaN,기타 미용업,857,2016-06-01 18:12:00,...,18,0,2,summer,NaN,NaN,제주,,0,0
2,0,2,c,2016-06-01,18:52,0,NaN,기타 미용업,2000,2016-06-01 18:52:00,...,18,0,2,summer,NaN,NaN,제주,,0,0
3,0,3,a,2016-06-01,20:22,0,NaN,기타 미용업,7857,2016-06-01 20:22:00,...,20,0,2,summer,NaN,NaN,제주,,0,0
4,1,1931,b,2016-06-01,13:31,0,NaN,NaN,1071,2016-06-01 13:31:00,...,13,0,2,summer,NaN,NaN,제주,,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6556608,2136,4659988,f,2017-10-04,23:32,0,제주 제주시,기타 주점업,9571,2017-10-04 23:32:00,...,23,0,4,fall,제주,제주시,제주,추석,1,1
6556609,2136,4659465,f,2017-10-04,23:41,0,제주 제주시,기타 주점업,7000,2017-10-04 23:41:00,...,23,0,4,fall,제주,제주시,제주,추석,1,1
6556610,2136,4659989,d,2017-10-04,23:47,0,제주 제주시,기타 주점업,5000,2017-10-04 23:47:00,...,23,0,4,fall,제주,제주시,제주,추석,1,1
6556611,2136,4659710,g,2017-10-04,23:53,0,제주 제주시,기타 주점업,3785,2017-10-04 23:53:00,...,23,0,4,fall,제주,제주시,제주,추석,1,1


In [27]:
def holidays_x(holi_sum):
    if holi_sum >=1:
        return "1"
    else:
        return "0"

df["holidays_x"] = df.holi_sum.apply(holidays_x)
df

,store_id,card_id,card_company,transacted_date,transacted_time,installment_term,region,type_of_business,amount,date,...,hour,holidays_x,quarter,season,sido,city,area,비고,holidays_y,holi_sum
0,0,0,b,2016-06-01,13:13,0,NaN,기타 미용업,1857,2016-06-01 13:13:00,...,13,0,2,summer,NaN,NaN,제주,,0,0
1,0,1,h,2016-06-01,18:12,0,NaN,기타 미용업,857,2016-06-01 18:12:00,...,18,0,2,summer,NaN,NaN,제주,,0,0
2,0,2,c,2016-06-01,18:52,0,NaN,기타 미용업,2000,2016-06-01 18:52:00,...,18,0,2,summer,NaN,NaN,제주,,0,0
3,0,3,a,2016-06-01,20:22,0,NaN,기타 미용업,7857,2016-06-01 20:22:00,...,20,0,2,summer,NaN,NaN,제주,,0,0
4,1,1931,b,2016-06-01,13:31,0,NaN,NaN,1071,2016-06-01 13:31:00,...,13,0,2,summer,NaN,NaN,제주,,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6556608,2136,4659988,f,2017-10-04,23:32,0,제주 제주시,기타 주점업,9571,2017-10-04 23:32:00,...,23,1,4,fall,제주,제주시,제주,추석,1,1
6556609,2136,4659465,f,2017-10-04,23:41,0,제주 제주시,기타 주점업,7000,2017-10-04 23:41:00,...,23,1,4,fall,제주,제주시,제주,추석,1,1
6556610,2136,4659989,d,2017-10-04,23:47,0,제주 제주시,기타 주점업,5000,2017-10-04 23:47:00,...,23,1,4,fall,제주,제주시,제주,추석,1,1
6556611,2136,4659710,g,2017-10-04,23:53,0,제주 제주시,기타 주점업,3785,2017-10-04 23:53:00,...,23,1,4,fall,제주,제주시,제주,추석,1,1


In [ ]:
df.columns

In [28]:
df = df.drop(['holidays_y', 'holi_sum'], axis = 1)

In [29]:
df.head(3)

,store_id,card_id,card_company,transacted_date,transacted_time,installment_term,region,type_of_business,amount,date,...,month,dayofweek,hour,holidays_x,quarter,season,sido,city,area,비고
0,0,0,b,2016-06-01,13:13,0,NaN,기타 미용업,1857,2016-06-01 13:13:00,...,6,2,13,0,2,summer,NaN,NaN,제주,
1,0,1,h,2016-06-01,18:12,0,NaN,기타 미용업,857,2016-06-01 18:12:00,...,6,2,18,0,2,summer,NaN,NaN,제주,
2,0,2,c,2016-06-01,18:52,0,NaN,기타 미용업,2000,2016-06-01 18:52:00,...,6,2,18,0,2,summer,NaN,NaN,제주,


## 결측치 제거

### region 과 type_of_business의 결측치
* region과 type_of_business 중에 1개라도 결측치면 제거
* 전체 데이터 중 약 40% 남음

In [30]:
# missing data
total = df.isnull().sum().sort_values(ascending =False)
percent = (df.isnull().sum()/df.isnull().count()).sort_values(ascending=False)
missing_data = pd.concat([total, percent], axis=1, keys=['Total','Percent'])
missing_data.head(10)

,Total,Percent
type_of_business,3952609,0.602843
city,2042766,0.311558
sido,2042766,0.311558
region,2042766,0.311558
store_id,0,0.000000
dayofweek,0,0.000000
area,0,0.000000
season,0,0.000000
quarter,0,0.000000
holidays_x,0,0.000000


In [31]:
df_0=df[df["region"].isnull() | df["type_of_business"].isnull()].index
df = df.drop(df_0)

In [32]:
df.isnull().sum()

store_id            0
card_id             0
card_company        0
transacted_date     0
transacted_time     0
installment_term    0
region              0
type_of_business    0
amount              0
date                0
year                0
month               0
dayofweek           0
hour                0
holidays_x          0
quarter             0
season              0
sido                0
city                0
area                0
비고                  0
dtype: int64

In [ ]:
# missing data check
total = df.isnull().sum().sort_values(ascending =False)
percent = (df.isnull().sum()/df.isnull().count()).sort_values(ascending=False)
missing_data = pd.concat([total, percent], axis=1, keys=['Total','Percent'])
missing_data.head(10)

In [33]:
df.shape

(2318410, 21)

In [ ]:
2318410 - 22

### (N/Y)amount 값 중  환불데이터가 아닌 음수인 값 제거

In [ ]:
test = df.groupby(['store_id','card_id'],)['amount'].sum().to_frame()
test

In [ ]:
test0 = test[test['amount']<0]
test0.index

In [ ]:
df = df.drop([5261])

In [ ]:
df = df.drop([1452443])
df = df.drop([5333077])
df = df.drop([6486])
df = df.drop([649141])
df = df.drop([5338432])
df = df.drop([62735])
df = df.drop([5334088])
df = df.drop([5107900])
df = df.drop([5107901])
df = df.drop([2814])
df = df.drop([5366637])
df = df.drop([105085])
df = df.drop([4790756])

In [ ]:
df = df.drop([4790757])
df = df.drop([23599])
df = df.drop([110286])
df = df.drop([5335461])
df = df.drop([5335462])
df = df.drop([9401])
df = df.drop([1847087])
df = df.drop([5345574])
df = df.drop([5345575])

In [ ]:
 test[test['amount']<0]

In [ ]:
df[(df["store_id"]==142) & (df["card_id"]==322529)] 

In [ ]:
df[(df["store_id"]==496) & (df["card_id"]==306712)] 

In [ ]:
df[(df["store_id"]==496) & (df["card_id"]==935481)] 

In [ ]:
df[(df["store_id"]==616) & (df["card_id"]==1197724)] 

In [ ]:
df[(df["store_id"]==770) & (df["card_id"]==1617416)] 

In [ ]:
df[(df["store_id"]==838) & (df["card_id"]==1785941)] 

In [ ]:
df[(df["store_id"]==838) & (df["card_id"]==1785956)] 

In [ ]:
df[(df["store_id"]==892) & (df["card_id"]==1896816)] 

In [ ]:
df[(df["store_id"]==977) & (df["card_id"]==2100492)] 

In [ ]:
df[(df["store_id"]==1100) & (df["card_id"]==2374816)] 

In [ ]:
df[(df["store_id"]==1354) & (df["card_id"]==2920340)] 

In [ ]:
df[(df["store_id"]==1366) & (df["card_id"]==2940184)] 

In [ ]:
df[(df["store_id"]==1538) & (df["card_id"]==3330482)]

In [ ]:
df[(df["store_id"]==1637) & (df["card_id"]==1537277)] 

In [ ]:
df[(df["store_id"]==1637) & (df["card_id"]==3549990)] 

In [ ]:
df[(df["store_id"]==1670) & (df["card_id"]==3593810)] 

In [ ]:
df[(df["store_id"]==1917) & (df["card_id"]==4138817)] 

In [ ]:
df[(df["store_id"]==1957) & (df["card_id"]==4232623)] 

In [ ]:
df[(df["store_id"]==2128) & (df["card_id"]==4644633)]

In [ ]:
[5345574]

In [ ]:
df[df.index == 5345574]

In [ ]:
df.shape

In [ ]:
2318388 -2318118

##  type_of_business 전처리

### 중분류 컬럼 만들기

In [37]:
# df 의 'type_of_business'와 'biz'파일의 띄어쓰기 다른 것 맞추기

In [34]:
df.loc[df["type_of_business"].str.contains("간이음식 포장 판매 전문점"),"type_of_business"]= "간이 음식 포장 판매 전문점"
df.loc[df["type_of_business"].str.contains("문구용품, 회화용품, 사무용품·· 도매업"),"type_of_business"]= "문구용품, 회화용품, 사무용품 도매업"
df.loc[df["type_of_business"].str.contains("통신·방송장비 및 부품 도매업"),"type_of_business"]= "통신ㆍ방송장비 및 부품 도매업"
df.loc[df["type_of_business"].str.contains("건강보조식품 소매업"),"type_of_business"]= "건강 보조식품 소매업"
df.loc[df["type_of_business"].str.contains("주방용품 및 가정용 유리, 요업 제품 소매업"),"type_of_business"]= "주방용품 및 가정용 유리, 요업제품 소매업"
df.loc[df["type_of_business"].str.contains("인물사진 및 행사용 영상 촬영업"),"type_of_business"]= "인물 사진 및 행사용 영상 촬영업"
df.loc[df["type_of_business"].str.contains("그 외 기타 분류 안된 사업지원 서비스업"),"type_of_business"]= "그 외 기타 분류 안된 사업 지원 서비스업"
df.loc[df["type_of_business"].str.contains("일반 교과 학원"),"type_of_business"]= "일반 교과학원"
df.loc[df["type_of_business"].str.contains("기타 기술 및 직업훈련학원"),"type_of_business"]= "기타 기술 및 직업 훈련학원"
df.loc[df["type_of_business"].str.contains("일반 병원"),"type_of_business"]= "일반병원"
df.loc[df["type_of_business"].str.contains("체력단련시설 운영업"),"type_of_business"]= "체력 단련시설 운영업"
df.loc[df["type_of_business"].str.contains("노래연습장 운영업"),"type_of_business"]= "노래 연습장 운영업"
df.loc[df["type_of_business"].str.contains("체형 등 기타 신체관리 서비스업"),"type_of_business"]= "체형 등 기타 신체 관리 서비스업"
df.loc[df["type_of_business"].str.contains("애완동물 장묘 및 보호 서비스업"),"type_of_business"]= "애완 동물 장묘 및 보호 서비스업"
df.loc[df["type_of_business"].str.contains("섬유, 직물 및 의복액세서리 소매업"),"type_of_business"]= "섬유, 직물 및 의복 액세서리 소매업"
df.loc[df["type_of_business"].str.contains("골프연습장 운영업"),"type_of_business"]= "골프 연습장 운영업"
df.loc[df["type_of_business"].str.contains("신선식품 및 단순 가공식품 도매업"),"type_of_business"]= "신선 식품 및 단순 가공 식품 도매업"

In [40]:
# 비즈파일 불러오기
biz = pd.read_csv('biz0.csv')
biz.head()

,개정 분류체계(제10차 기준),Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9
0,대분류(21),NaN,중분류(77),NaN,소분류(232),NaN,세분류(495),NaN,"세세분류(1,196)",NaN
1,코드,항목명,코드,항목명,코드,항목명,코드,항목명,코드,항목명
2,A,"농업, 임업 및 어업(01~03)",01,농업,011,작물 재배업,0111,곡물 및 기타 식량작물 재배업,01110,곡물 및 기타 식량작물 재배업
3,NaN,NaN,NaN,NaN,NaN,NaN,0112,"채소, 화훼작물 및 종묘 재배업",01121,채소작물 재배업
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,01122,화훼작물 재배업


In [42]:
# 결측치 위에 데이터로 채우기
biz = biz.fillna(method='ffill')
biz

,개정 분류체계(제10차 기준),Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9
0,대분류(21),NaN,중분류(77),NaN,소분류(232),NaN,세분류(495),NaN,"세세분류(1,196)",NaN
1,코드,항목명,코드,항목명,코드,항목명,코드,항목명,코드,항목명
2,A,"농업, 임업 및 어업(01~03)",01,농업,011,작물 재배업,0111,곡물 및 기타 식량작물 재배업,01110,곡물 및 기타 식량작물 재배업
3,A,"농업, 임업 및 어업(01~03)",01,농업,011,작물 재배업,0112,"채소, 화훼작물 및 종묘 재배업",01121,채소작물 재배업
4,A,"농업, 임업 및 어업(01~03)",01,농업,011,작물 재배업,0112,"채소, 화훼작물 및 종묘 재배업",01122,화훼작물 재배업
...,...,...,...,...,...,...,...,...,...,...
2391,T,가구 내 고용활동 및 달리 분류되지 않은 자가 소비 생산활동(97~98),97,가구 내 고용활동,970,가구 내 고용활동,9700,가구 내 고용활동,9700,가구 내 고용활동
2392,T,가구 내 고용활동 및 달리 분류되지 않은 자가 소비 생산활동(97~98),98,달리 분류되지 않은 자가 소비를 위한 가구의 재화 및 서비스 생산활동,981,자가 소비를 위한 가사 생산 활동,9810,자가 소비를 위한 가사 생산 활동,9810,자가 소비를 위한 가사 생산 활동
2393,T,가구 내 고용활동 및 달리 분류되지 않은 자가 소비 생산활동(97~98),98,달리 분류되지 않은 자가 소비를 위한 가구의 재화 및 서비스 생산활동,982,자가 소비를 위한 가사 서비스 활동,9820,자가 소비를 위한 가사 서비스 활동,9820,자가 소비를 위한 가사 서비스 활동
2394,U,국제 및 외국기관(99),99,국제 및 외국기관,990,국제 및 외국기관,9900,국제 및 외국기관,9900,국제 및 외국기관


In [46]:
biz[biz ['Unnamed: 9'] == '한식 음식점업']

,개정 분류체계(제10차 기준),Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9
2034,I,숙박 및 음식점업(55~56),56,음식점 및 주점업,561,음식점업,5611,한식 음식점업,5611,한식 음식점업
2035,I,숙박 및 음식점업(55~56),56,음식점 및 주점업,561,음식점업,5611,한식 음식점업,5611,한식 음식점업
2036,I,숙박 및 음식점업(55~56),56,음식점 및 주점업,561,음식점업,5611,한식 음식점업,5611,한식 음식점업
2037,I,숙박 및 음식점업(55~56),56,음식점 및 주점업,561,음식점업,5611,한식 음식점업,5611,한식 음식점업


In [ ]:
biz.head(4)

In [ ]:
biz.columns

In [ ]:
biz2 = biz.drop(['개정 분류체계(제10차 기준)', 'Unnamed: 2','Unnamed: 4','Unnamed: 6','Unnamed: 8',], axis = 1)

In [ ]:
biz2.to_excel('funda_biz.xlsx', index= False)

In [ ]:
biz2.to_csv('biz.csv', index= False)

In [ ]:
biz.rename(columns={'Unnamed: 9':'type_of_business'}, inplace=True)

In [ ]:
biz['type_of_business'].unique()

In [ ]:
biz = pd.read_csv('biz.csv')
biz

In [ ]:
biz

In [ ]:
for i in range(len(df)):
    for j in range(len(biz)):
        if df['type_of_business'][i] == biz['Unnamed: 9'][j] :
            df['type_of_business'][i] = biz['Unnamed: 1'][j] :
                
                

## 데이터 저장 

In [ ]:
df.to_csv('funda_data1.csv', index= False)

In [ ]:
df=pd.read_csv('funda_data1.csv')
df

In [ ]:
df.shape